In [1]:
import bs4 as bs, urllib, pandas as pd, numpy as np

Test parsing syntax

In [2]:
keyword='farkas'
time1='2018-07-01'
time2='2018-08-01'
baseurl=u'https://szekelyhon.ro/kereses?op=search&src_words='
url=baseurl+keyword+'&src_time1='+time1+'&src_time2='+time2

Parse past X years

In [5]:
start='2002-12-01'
end='2018-11-01'
dates=[]
datelist = pd.date_range(start=pd.to_datetime(start), end=pd.to_datetime(end), freq='M').tolist()
for date in datelist:
    dates.append(str(date)[:10])

In [6]:
def extractor(time1,time2):
    time1=dates[i]
    time2=dates[i+1]
    print('Parsing...',time1,'-',time2)
    url=baseurl+keyword+'&src_time1='+time1+'&src_time2='+time2
    html = urllib.request.urlopen(url).read()
    soup = bs.BeautifulSoup(html,'lxml')
    return soup.findAll("div", {"class": "catinner"})

In [7]:
divs=[]
for i in range(len(dates)-1):
    time1=dates[i]
    time2=dates[i+1]
    divs.append(extractor(time1,time2))

Parsing... 2002-12-31 - 2003-01-31
Parsing... 2003-01-31 - 2003-02-28
Parsing... 2003-02-28 - 2003-03-31
Parsing... 2003-03-31 - 2003-04-30
Parsing... 2003-04-30 - 2003-05-31
Parsing... 2003-05-31 - 2003-06-30
Parsing... 2003-06-30 - 2003-07-31
Parsing... 2003-07-31 - 2003-08-31
Parsing... 2003-08-31 - 2003-09-30
Parsing... 2003-09-30 - 2003-10-31
Parsing... 2003-10-31 - 2003-11-30
Parsing... 2003-11-30 - 2003-12-31
Parsing... 2003-12-31 - 2004-01-31
Parsing... 2004-01-31 - 2004-02-29
Parsing... 2004-02-29 - 2004-03-31
Parsing... 2004-03-31 - 2004-04-30
Parsing... 2004-04-30 - 2004-05-31
Parsing... 2004-05-31 - 2004-06-30
Parsing... 2004-06-30 - 2004-07-31
Parsing... 2004-07-31 - 2004-08-31
Parsing... 2004-08-31 - 2004-09-30
Parsing... 2004-09-30 - 2004-10-31
Parsing... 2004-10-31 - 2004-11-30
Parsing... 2004-11-30 - 2004-12-31
Parsing... 2004-12-31 - 2005-01-31
Parsing... 2005-01-31 - 2005-02-28
Parsing... 2005-02-28 - 2005-03-31
Parsing... 2005-03-31 - 2005-04-30
Parsing... 2005-04-3

In [8]:
def date_hu_en(i):
    date=i[6:-4]
    if date=='augusztus': m='08'
    elif date=='december': m='12'
    elif date=='február': m='02'
    elif date=='január': m='01'
    elif date=='július': m='07'
    elif date=='június': m='06'
    elif date=='május': m='05'
    elif date=='március': m='03'
    elif date=='november': m='11'
    elif date==u'október': m='10'
    elif date==u'szeptember': m='09'
    elif date==u'április': m='04'
    else: return date
    return i[:4]+'-'+m+'-'+i[-3:-1]

Relevant = Medves cikk-e vagy sem  
Deaths = Halalok szama  
Severity = Sulyossag: 0-mas jellegu hir, 1-latas, 2-allat-tamadas, 3-ember-tamadas

In [16]:
def text_processor(title,content):
    relevant=0
    severity=0
    deaths=0
    tamadas=[u'támad',u'sebes']
    for i in tamadas:
        if i in title+content:
            relevant=1
            severity=2
    tamadas=[u'halál',u'áldozat',u'ölt ']
    for i in tamadas:
        if i in title+content:
            relevant=1
            severity=3
    tamadas=[u'farkas']
    for i in tamadas:
        if i in title.replace(',',' ').replace('.',' ').lower():
            relevant=1
    return relevant,severity,deaths

In [17]:
hirek=[]
for divgroup in divs:
    for div in divgroup:
        idiv=div.find('div')
        content=div.find('p')
        date=idiv.text[idiv.text.find('201'):idiv.text.find(',')]
        title=div.find('h2').text
        if content==None:
            sdiv=str(div)[::-1]
            content=sdiv[:sdiv.find('>a/<')].replace('\r','').replace('\t','').replace('\n','')[::-1][:-6]
        else: content=content.text
        relevant,severity,deaths=text_processor(title,content)
        hirek.append({'date':date_hu_en(date),
                      'hudate':date,
                      'title':title,
                      'content':content,
                      'link':div.findAll('a')[-1]['href'],
                      'relevant':relevant,
                      'severity':severity,
                      'deaths':deaths
                       })

Összes farkasos hír

In [18]:
len(hirek)

2282

In [19]:
df=pd.DataFrame().from_dict(hirek)
df['date']=pd.to_datetime(df['date'])
df=df.sort_values('date').reset_index(drop=True)

Save to medve Excel. Manual curation

In [20]:
df.columns

Index(['content', 'date', 'deaths', 'hudate', 'link', 'relevant', 'severity',
       'title'],
      dtype='object')

In [21]:
df=df[[ 'date', 'hudate', 'link', 'title','content','relevant', 'severity','deaths']]

In [22]:
df.to_excel('szekelyhon_farkas.xlsx')
df.to_excel('szekelyhon_farkas_curated.xlsx')

Manually check and curate the `curated` file. Don't touch the other one.  
Load curated

In [327]:
dc=pd.read_excel('szekelyhon_farkas_curated.xlsx')

In [330]:
dg=dc[dc['relevant']==1]

In [333]:
dg

,date,hudate,link,title,content,relevant,severity,deaths,duplicate
68,2010-05-17,2010. május 17.,https://szekelyhon.ro/aktualis/medve-latogatta...,medve látogatta meg a Szent Anna-tónál sátrazókat,Szombatról vasárnapra virradó éjszaka a 112-es...,1,1,0,0
82,2010-06-22,2010. június 22.,https://szekelyhon.ro/aktualis/medve-emberlesen,medve emberlesen,Ismét lakóterületre merészkednek a medvék – a ...,1,1,0,0
86,2010-06-23,2010. június 23.,https://szekelyhon.ro/,Bikfalvi medveterror,Valósággal rettegésben tartja a Kovászna megye...,1,1,0,0
91,2010-06-30,2010. június 30.,https://szekelyhon.ro/sport/egyeb-sportok/medv...,medvelátogatás a Szent Anna-tó melletti sátort...,Idén sem maradt el a hagyományosan a Szent Ann...,1,1,0,0
97,2010-07-09,2010. július 09.,https://szekelyhon.ro/,Jogköreik bővítését kérik a székelyföldi erdőt...,Több mint 150 Hargita és Kovászna megyei közbi...,1,0,0,0
99,2010-07-20,2010. július 20.,https://szekelyhon.ro/aktualis/juhokat-olt-a-m...,Juhokat ölt a medve Tekerőpatakon,medve pusztított hétfőről keddre virradóra töb...,1,2,0,0
101,2010-07-26,2010. július 26.,https://szekelyhon.ro/aktualis/gyergyoszek/gye...,Gyergyószentmiklóson is juhokat ölt a medve,Rettegnek Gyergyószentmiklós felszegi részének...,1,2,0,0
102,2010-07-26,2010. július 26.,https://szekelyhon.ro/aktualis/a-kisallatok-is...,A kisállatok is okozhatnak károkat a gazdáknak,"Róka, görény, borz – nemcsak a medve és a vadd...",1,0,0,0
103,2010-07-27,2010. július 27.,https://szekelyhon.ro/aktualis/ujra-bejartak-a...,Újra bejártak a medvék Gyergyószentmiklósra,A hétfő hajnali medvetámadás megismétlődésétől...,1,1,0,0
105,2010-07-28,2010. július 28.,https://szekelyhon.ro/aktualis/vitathato-az-op...,Vitatható az optimális medveszám,Az utóbbi évtizedben folyamatosan emelkedett H...,1,0,0,0


Further fixes from full text

In [337]:
hirek_full=[]
for divgroup in divs:
    for div in divgroup:
        hirek_full.append(div)
        

In [338]:
df=pd.DataFrame().from_dict(hirek)
df['date']=pd.to_datetime(df['date'])
df=df.sort_values('date').reset_index()

In [340]:
index_converter=list(df['index'])

In [344]:
hirek[index_converter[252]]

{'date': '2011-05-23',
 'hudate': '2011. május 23.',
 'title': ' Újabb medvetámadás Háromszéken ',
 'content': 'Medve támadt egy erdővidéki férfire, az anyamedve a Barót melletti erdőkben rontott rá az 50 éves férfira, és összemarcangolta a felsőtestét. ',
 'link': 'https://szekelyhon.ro/',
 'relevant': 1,
 'severity': 2,
 'deaths': 0}

In [345]:
hirek[index_converter[1987]]

{'date': '',
 'hudate': '',
 'title': 'Székely medveriadó',
 'content': 'Élelmet kereső medvék garázdálkodnak a székelyföldi településeken az ott élők és a környékre látogató turisták rémületére. A jelenség aggodalomra ad okot, mivel eddig errefelé nem volt példa rá, hogy a medvék nappal is lemerészkedjenek a településekre. A vadállatok egyelőre nem veszélyeztették az emberek testi épségét, Gyergyószéken azonban napi rendszerességgel tizedelik a háziállat-állományt. A „medvelátogatásokról” egyébként meglehetősen...',
 'link': 'https://szekelyhon.ro/',
 'relevant': 1,
 'severity': 0,
 'deaths': 0}